In [ ]:
# 5.7.1 VGG块
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn
# VGG块连续使用数个相同的填充为1、窗口形状为 3×3 的卷积层后接上一个步幅为2、窗口形状为 2×2 的最大池化层。
def vgg_block(num_convs,num_channels):
    #num_convs为卷积层的数量 num_channels为输出通道数
    blk=nn.Sequential()
    for _ in range(num_convs):
        # 卷积层保持输⼊的⾼和宽不变(n-3+2+1)
        blk.add(nn.Conv2D(num_channels,kernel_size=3,padding=1,activation='relu'))
    # 池化层则对高和宽减半(n-2+2)/2
    blk.add(nn.MaxPool2D(pool_size=2,strides=2))
    return blk

In [ ]:
# 5.7.2 VGG⽹络 
# 与AlexNet和LeNet⼀样，VGG⽹络由卷积层模块后接全连接层模块构成。
# 卷积层模块串联数个vgg_block，其超参数由变量conv_arch定义。该变量指定了每个VGG块⾥卷积层个数和输出通道数。
conv_arch=((1,64),(1,128),(2,256),(2,512),(2,512))
# VGG-11
def vgg(conv_arch):
    net=nn.Sequential()
    # 卷积层部分
    for (num_convs,num_channels) in conv_arch:
        net.add(vgg_block(num_convs,num_channels))
    # 全连接层部分
    net.add(nn.Dense(4096,activation='relu'),nn.Dropout(0.5),
           nn.Dense(4096,activation='relu'),nn.Dropout(0.5),
           nn.Dense(10))
    return net

net=vgg(conv_arch)

In [23]:
net.initialize()
X=nd.random.uniform(shape=(1,1,224,224))
for blk in net:
    X=blk(X)
    print(blk.name,'output shape:\t',X.shape)

sequential44 output shape:	 (1, 64, 112, 112)
sequential45 output shape:	 (1, 128, 56, 56)
sequential46 output shape:	 (1, 256, 28, 28)
sequential47 output shape:	 (1, 512, 14, 14)
sequential48 output shape:	 (1, 512, 7, 7)
dense21 output shape:	 (1, 4096)
dropout14 output shape:	 (1, 4096)
dense22 output shape:	 (1, 4096)
dropout15 output shape:	 (1, 4096)
dense23 output shape:	 (1, 10)


In [26]:
# 5.7.3 获取数据和训练模型 
ratio=4
small_conv_arch=[(pair[0],pair[1]//ratio) for pair in conv_arch]
net=vgg(small_conv_arch)# 构造⼀个通道数更小的网络

In [ ]:
# 与AlexNet相比使用了稍大些的学习率，模型训练过程与AlexNet类似。
import mxnet as mx
lr,num_epochs,batch_size,ctx=0.05,5,128,mx.cpu()
net.initialize(ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=224)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on cpu(0)


In [29]:
d2l.train_ch5??
d2l.load_data_fashion_mnist??